In [1]:
%matplotlib inline
import numpy as np
import matplotlib.tri as Tri
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset as NetCDFFile 
import netCDF4
from netCDF4 import num2date, date2num, date2index
import datetime as dt
import pandas as pd
from StringIO import StringIO
import xarray as xr

/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
import re

def extractURLsatellite(fileURL, satName):
    """
    Function to extract the URLs for a specific satellite from the IMOS URLs list
    
    Parameters:
    -----------
    
    - fileURL : IMOS URLs list as a txt file
    - satName : name of the satellite such as JASON-2 JASON-3 
    
    Ouputs:
    ------
    
    - getFiles : list of URLs for the desired satellite
    
    """
    
    getFiles = []
    
    with open(fileURL) as f:
        for line in f:
            if re.search(r"%s"%satName, line):
                changeURL = re.sub('http://data.aodn.org.au', 'http://thredds.aodn.org.au/thredds/dodsC', line)
                getFiles.append(changeURL)
                
    return getFiles

In [4]:
jason2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-2')

jason3URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-3')

saralURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SARAL')

sentinel3aURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SENTINEL-3A')

cryosat2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'CRYOSAT-2')

enviURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ENVISAT')

geosatURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GEOSAT')

ersURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ERS-2')

gfoURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GFO')

topURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'TOPEX')

In [5]:
allURL = []
allURL.append(jason2URL)
allURL.append(jason3URL)
allURL.append(saralURL)
allURL.append(sentinel3aURL)
allURL.append(cryosat2URL)
allURL.append(enviURL)
allURL.append(geosatURL)
allURL.append(ersURL)
allURL.append(gfoURL)
allURL.append(topURL)

boxLat = [] #latitude
boxLon = [] #longitude
boxWh = [] #wave height
boxT = [] #time
boxQ = [] #quality control flag
boxB = [] #backscatter
boxWS = [] #wind Speed

In [6]:
latmin = -18.0
latmax = -17.0  
lonmin = 147.0
lonmax = 148.0 

if latmin>latmax:
    print('Error wrong definition of min and max lat!!!')

if lonmin>lonmax:
    print('Error wrong definition of min and max lon!!!')



start_date = dt.datetime(1985,1,1)
end_date = dt.datetime(2019,2,21)
 
#start_date = dt.datetime(2010,1,1)
#end_date = dt.datetime(2010,12,31)


In [7]:
boxLat = [] #latitude
boxLon = [] #longitude
boxWh = [] #wave height
boxT = [] #time
boxQ = [] #quality control flag
boxB = [] #backscatter
boxWS = [] #wind Speed

#urlON = allURL
#print(len(urlON))
newday_it = 0
#for k in range(164,165):

for u in range(len(allURL)):
    urlON = allURL[u]
    for k in range (len(urlON)):
        ncs = NetCDFFile(urlON[k])
        lats = ncs.variables['LATITUDE'][:]
        lons = ncs.variables['LONGITUDE'][:]
        ws = ncs.variables['WSPD_CAL'][:]
        
        
        if u == 2:
            wh = ncs.variables['SWH_KA_CAL'][:]
            qc = ncs.variables['SWH_KA_quality_control'][:]
            back = ncs.variables['SIG0_KA'][:]
        
        else:
            wh = ncs.variables['SWH_KU_CAL'][:]
            qc = ncs.variables['SWH_KU_quality_control'][:]
            back = ncs.variables['SIG0_KU'][:]
            
            
            # Get desired time interval  
            time_var = ncs.variables['TIME']
            tt = ncs.variables['TIME'][:]
            timing = netCDF4.num2date(tt,time_var.units)

            #print(len(lats),len(ws),len(timing),len(qc),qc.min(),qc.max())
            newday_it = 0
            if lats.min() >= latmin and lats.min() <= latmax:
                if lons.min() >= lonmin and lons.min() <= lonmax:
                    for p in range(len(timing)):
                        if timing[p] >= start_date and timing[p] <= end_date:
                            if wh[p]>0 and qc[p]==1:

                                t1 = netCDF4.num2date(tt[p],u'days since 1985-01-01 00:00:00 UTC')
        #                         print t1
                                if newday_it == 0:
                                    dd = netCDF4.num2date(tt[p],u'days since 1985-01-01 00:00:00 UTC')
                                    newday_it = p
                                else:
                                    if t1.day != dd.day:
                                        boxLat.append(np.mean(lats[newday_it:p]))   
        #                                 print newday_it,p
                                        
                                        dd = netCDF4.num2date(tt[p],u'days since 1985-01-01 00:00:00 UTC')
                                        boxLon.append(np.mean(lons[newday_it:p]))
                                        boxWh.append(np.mean(wh[newday_it:p]))
                                        boxT.append(np.mean(tt[newday_it:p]))
                                        boxB.append(np.mean(back[newday_it:p]))
                                        boxWS.append(np.mean(ws[newday_it:p]))
                                        boxQ.append(np.mean(qc[newday_it:p]))   
                                        newday_it = p

        #                 print(lats.min(),lats.max(),lons.min(),lons.max())
    #                 if timing[p] >= start_date and timing[p] <= end_date:
    #                     print 
    #     if latmin >= lats.min() and latmax <= lats.min():
#     for p in range(len(timing)):
#         if timing[p] >= start_date and timing[p] <= end_date:
#             if lats[p]>latmin and lats[p]<latmax and lons[p]>lonmin and lons[p]<lonmax:
print len(boxLat)

2662


In [8]:
max(boxQ)
len(boxLat)

2662

In [9]:
for k in range(len(boxLat)):
    if k == 0:
        lat = boxLat[k]
        lon = boxLon[k]
        wh = boxWh[k]
        tt = boxT[k]
        qc = boxQ[k]
        back = boxB[k]
        ws = boxWS[k]
    else:
        lat = np.append(lat,boxLat[k])
        lon = np.append(lon,boxLon[k])
        wh = np.append(wh,boxWh[k])
        tt = np.append(tt,boxT[k])
        qc = np.append(qc,boxQ[k])
        back = np.append(back,boxB[k])
        ws = np.append(ws,boxWS[k])

In [10]:
df = pd.DataFrame(
    {'lat':lat.flatten(),
     'lon':lon.flatten(),
     'wh':wh.flatten(),
     'tt':tt.flatten(),
     'qc':qc.flatten(),
     'back':back.flatten(),
     'ws':ws.flatten()
    })
nameCSV = 'innis_1.csv'
df.to_csv(str(nameCSV),columns=['lat', 'lon', 'wh', 'tt', 'qc', 'back','ws'], sep=' ', index=False ,header=1)

In [ ]:
len(qc)